In [64]:
import numpy as np
A = np.eye(10) + np.eye(10, k=1)*2 + np.eye(10, k=-1)*0.5
b = np.ones(10)
x = np.dot(np.linalg.inv(A), b)
print("Ax=b")
print("行列")
print(A)
print("右辺")
print(b)
print("解")
print(x)

Ax=b
行列
[[1.  2.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.9 1.  2.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.9 1.  2.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.9 1.  2.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.9 1.  2.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.9 1.  2.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.9 1.  2.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.9 1.  2.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.9 1.  2. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.9 1. ]]
右辺
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
解
[-26.30364996  13.65182498   5.51072999  -8.39868624   2.21951462
   3.1696515   -2.08360733   0.11546049   1.37989305  -0.24190375]


# BiCG Method

In [71]:
# Initialize solution x
x_bicg = np.ones(len(x))
A_temp = A.copy()
b_base = b.copy()
b_temp = np.dot(A_temp, x_bicg)
r = b_base - b_temp
p = r.copy()
ps = r.copy()
rs = r.copy()
r0 = np.sqrt(np.dot(r, r))
eps = 10 ** -10
for i in range(10000):
    # q _k= Ap_k
    b_temp = np.dot(A_temp, p)
    # q* _k= AHp*_k
    b_temp2 = np.dot(np.linalg.inv(A_temp), ps)
    # (r*_k,r_k)
    t = np.dot(rs, r)
    # alpha_k = (r*_k,r_k) / (p*_k,q_k)
    alpha = t / np.dot(ps, b_temp)
    # x_k1 = x_k + alpha_k * p_k
    x_bicg += alpha * p
    # r_k1 = r_k - alpha_k * q_k
    r -= alpha * b_temp
    # r*_k1 = r*_k - alpha_k * q*_k
    rs -= alpha * b_temp2
    # (r*_k1, r_k1) / (r*_k, r_k)
    beta = np.dot(rs, r) / t
    # p_k1 = r_k1 + beta_k * p_k
    p = r + beta * p
    # p*_k1 = r*_k1 + beta_k * p*_k
    ps = rs + beta * ps
    if i % 1000 == 0:
        print(i, np.sum(x_bicg - x))
    if np.sqrt(np.dot(r, r)) / r0 <= eps:
        print(i, x_bicg - x)
        break
print(np.dot(A, x_bicg))

0 13.939100048354486
1000 13.814035692675501
2000 13.814035692675501
3000 13.814035692675501
4000 13.814035692675501
5000 13.814035692675501
6000 13.814035692675501
7000 13.814035692675501
8000 13.814035692675501
9000 13.814035692675501
[ 1.35132883  1.05255673  1.13577415  0.95174909  1.34142907  0.65818915
 -1.61156594  2.55546585  1.23926597  1.77499363]


# BiCGstab

In [75]:
# Initialize solution x
x_bicgstab = np.ones(len(x))
A_temp = A.copy()
b_base = b.copy()
b_temp = np.dot(A_temp, x_bicgstab)
r = b_base - b_temp
p = r.copy()
rs = r.copy()
r0 = np.sqrt(np.dot(r, r))
eps = 10 ** -10
for i in range(10000):
    # q _k= Ap_k
    b_temp = np.dot(A_temp, p)
    # (r*_k,r_k)
    t = np.dot(rs, r)
    # alpha_k = (r*_k,r_k) / (p*_k,q_k)
    alpha = t / np.dot(rs, b_temp)
    # s_k = r_k - alpha_k * q_k
    s = r - alpha * b_temp
    # w_k = (As_k, s_k) / (As_
    As = np.dot(A, s)
    w = np.dot(As, s) / np.dot(As, As)
    # x_k1 = x_k + alpha_k * p_k
    x_bicgstab += alpha * p + w*s
    # r_k1 = s_k - w_k * As_k
    r = s - w * As
    # (r*_k, r_k1) / (r*_k, r_k) * alpha_k / w_k
    beta = np.dot(rs, r) / t * alpha / w
    # p_k1 = r_k1 + beta_k * (p_k - w_k * Ap_k)
    p = r + beta * (p - w * np.dot(A, p)) 
    if i % 1000 == 0:
        print(i, np.sum(x_bicgstab - x))
    if np.sqrt(np.dot(r, r)) / r0 <= eps:
        print(i, x_bicgstab - x)
        print("residual: ", np.sqrt(np.dot(r, r)))
        break
print(np.dot(A, x_bicgstab))

0 13.94858455925076
15 [-1.34796991e-08  6.84646473e-09  2.60055177e-09 -4.40509851e-09
  1.06258868e-09  1.42021284e-09 -1.17822951e-09  2.48688292e-11
  4.33196590e-10 -2.55917926e-10]
residual:  3.671937470935679e-10
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [76]:
import numpy.linalg as nl
# initialize
x_j = np.zeros(b.size)
r_j = r_0 = p_j = b - A.dot(x_j)
r_0_norm = nl.norm(r_0)

# start
for j in range(1, 100001):
    Ap = A.dot(p_j)
    t = np.dot(r_j,r_0)
    a = t/np.dot(Ap,r_0)
    s = r_j - a*Ap
    As = A.dot(s)
    w = np.dot(As,s)/np.dot(As,As)
    x_j = x_j + a*p_j + w*s
    r_j = s - w*As
    c = np.dot(r_j,r_0)/t*a/w
    p_j = r_j + c*(p_j - w*Ap)

    #convergence test
    #print nl.norm(r_j)/r_0_norm
    if np.dot(r_j,r_j) < eps*eps*r_0_norm*r_0_norm:
        print('#number of iterations:', j)
        print('#relative residual: %e' % (nl.norm(r_j)/r_0_norm))
        print('#relative error:    %e' % (nl.norm(b-A.dot(x_j))/r_0_norm))
        break
else:
    print('do not converge!')

#number of iterations: 22
#relative residual: 2.629025e-11
#relative error:    2.628939e-11


In [77]:
np.dot(A, x_j)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])